# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import pandas as pd
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
keyspace = 'udacity'

session.execute("""
CREATE KEYSPACE IF NOT EXISTS %s
WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
""" % keyspace);

#### Set Keyspace

In [ ]:
session.set_keyspace(keyspace)

#### Utils functions

In [ ]:
def row_from_line(columns, fields):
    """
    Function that uses the column names and a list of fields (of a single row) to create a 
    python dictionary whose keys are the column names and values are the corresponding field 
    values.
    The function casts types based on the data type of the column. The mapping is hard-coded
    in this function.
    
    :param columns: List of strings that describe the header
    :param fields: List of fields within a single row (each item is a string).
    """
    ret = {}
    for i, c in enumerate(columns):
        if c in ['artist', 'firstName', 'gender', 'lastName', 'level', 'location', 'song']:
            ret[c] = fields[i]
        elif c in ['itemInSession', 'sessionId', 'userId']:
            ret[c] = int(fields[i])
        elif c in ['length']:
            ret[c] = float(fields[i])
        else:
            raise RuntimeError("Field '{}' is unknown and cannot be parsed.".format(c))
    return ret

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Exercise 1

The table `song_info_by_session` uses the `session_id` as its partition key and `item_in_session` as its clustering key.
It also stores the fields `artist`, `song` and `length` and thus can answer question like: 
> Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

All variables related to exercise 1 are prefixed with `ex1_`.

In [ ]:
ex1_create = session.prepare("""
CREATE TABLE IF NOT EXISTS song_info_by_session (
    session_id int,
    item_in_session int,
    artist text,
    song text,
    length float,
    PRIMARY KEY (session_id, item_in_session)
)
""")
ex1_create.__doc__ = """

"""

# Create database table
session.execute(ex1_create);

ex1_query = session.prepare("""
SELECT artist, song, length
FROM song_info_by_session
WHERE session_id = ? 
    AND item_in_session = ?
""")

ex1_insert = session.prepare("""
INSERT INTO song_info_by_session (session_id, item_in_session, artist, song, length)
VALUES (?, ?, ?, ?, ?)
""")

### Exercise 2

The table `song_and_user_info_by_user_and_session` uses the `user_id` and `session_id`as its partition key. It uses `item_in_session` as its clustering key (for sorting).
The table also stores the fields `artist`, `song` and `user` (which is composed from `first_name` and `last_name`) and thus can answer question like: 
> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

All variables related to exercise 2 are prefixed with `ex2_`.

In [ ]:
## Query 2: 

ex2_create = session.prepare("""
CREATE TABLE IF NOT EXISTS song_and_user_info_by_user_and_session (
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    song text,
    user text,
    PRIMARY KEY ((user_id, session_id), item_in_session)
)
""")

# Create database table
session.execute(ex2_create);

ex2_query = session.prepare("""
SELECT artist, song, user
FROM song_and_user_info_by_user_and_session
WHERE user_id = ? 
    AND session_id = ?
""")

ex2_insert = session.prepare("""
INSERT INTO song_and_user_info_by_user_and_session (user_id, session_id, item_in_session, artist, song, user)
VALUES (?, ?, ?, ?, ?, ?)
""")

### Exercise 3

The table `user_info_by_song_title` uses `song` (text) as its partition key. It also utilizes `user_id` as its clustering key, which is needed in case many users listened to the same song (uniqueness).
The table also stores the fields `first_name` and `last_name` and thus can answer question like: 
> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

All variables related to exercise 3 are prefixed with `ex3_`.

In [ ]:
ex3_create = session.prepare("""
CREATE TABLE IF NOT EXISTS user_info_by_song_title (
    song text,
    user_id int,
    first_name text,
    last_name text,
    PRIMARY KEY ((song), user_id)
)
""")

# Create database table
session.execute(ex3_create);

ex3_query = session.prepare("""
SELECT first_name, last_name
FROM user_info_by_song_title
WHERE song = ?
""")

ex3_insert = session.prepare("""
INSERT INTO user_info_by_song_title (song, user_id, first_name, last_name)
VALUES (?, ?, ?, ?)
""")

## Process and insert values

In this step we read the CSV file and insert data in all three tables in a single pass.

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    header = next(csvreader)
    for line in csvreader:
        row = row_from_line(header, line)
        user = '{} {}'.format(row['firstName'], row['lastName'])
        session.execute(ex1_insert, (row['sessionId'], row['itemInSession'], row['artist'], row['song'], row['length']))
        session.execute(ex2_insert, (row['userId'], row['sessionId'], row['itemInSession'], row['artist'], row['song'], user))
        session.execute(ex3_insert, (row['song'], row['userId'], row['firstName'], row['lastName']))

## Answers

This section gives answers to the following questions:

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
## Query 1
rows = session.execute(ex1_query, (338, 4))

pd.DataFrame(list(rows))

In [ ]:
# Query 2
rows = session.execute(ex2_query, (10, 182))

pd.DataFrame(list(rows))

In [ ]:
## Query 3
rows = session.execute(ex3_query, ('All Hands Against His Own',))

pd.DataFrame(list(rows))

### Drop the tables before closing out the sessions

In [ ]:
ex1_delete = session.prepare("DROP TABLE IF EXISTS song_info_by_session");
ex2_delete = session.prepare("DROP TABLE IF EXISTS song_and_user_info_by_user_and_session");
ex3_delete = session.prepare("DROP TABLE IF EXISTS user_info_by_song_title");

In [ ]:
for query in [ex1_delete, ex2_delete, ex3_delete]:
    session.execute(query);

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()